In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK：AutoML 表格二进制分类模型的批量解释

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_automl_tabular_binary_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexplainable_ai%2Fsdk_automl_tabular_binary_classification_batch_explain.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/explainable_ai/sdk_automl_tabular_binary_classification_batch_explain.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_automl_tabular_binary_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Vertex AI SDK创建一个表格二分类模型，并使用Google Cloud进行批量预测并提供解释。了解更多关于[自动化机器学习(AutoML)](https://cloud.google.com/vertex-ai/docs/start/automl-users)的内容。

了解更多关于[表格数据分类](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview)。了解更多关于[Vertex可解释性人工智能](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)。

### 目标

在本教程中，您将学习如何使用`AutoML`从Python脚本创建一个表格二元分类模型，然后学习如何使用`Vertex AI批量预测`进行带有解释的预测。您还可以选择使用`gcloud`命令行工具或在线使用Cloud Console来创建和部署模型。

本教程使用以下Vertex AI服务：

- Vertex AI AutoML
- Vertex AI Batch Prediction
- Vertex可解释 AI
- Vertex AI模型资源

执行的步骤包括：

- 创建一个Vertex AI托管的数据集资源。
- 训练一个AutoML表格二元分类模型。
- 查看训练模型的模型评估指标。
- 进行带有解释的批量预测请求。

使用批量预测和在线预测之间有一个关键区别：

* **预测服务**：为整个实例集（即一个或多个数据项）执行按需预测，并实时返回结果。

* **批量预测服务**：对整个实例集进行排队（批量）预测，后台存储结果，并在准备就绪时将结果存储在Cloud存储桶中。

### 数据集

本教程使用了一个银行营销数据集。这个数据集不需要任何特征工程。在本教程中使用的数据集版本存储在一个公共云存储桶中。

### 成本

本教程使用Google Cloud的计费组件：

- Vertex AI
- Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)基于您的预期使用量生成成本估算。

开始吧

### 为 Python 安装 Vertex AI SDK 和其他所需的包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform
! pip3 install --upgrade --quiet google-cloud-storage
! pip3 install --upgrade --quiet tensorflow

### 重新启动运行时（仅适用于Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。请等待直到完成再继续下一步。⚠️</b>
</div>

### 在谷歌Colab上验证您的笔记本环境

在谷歌Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 初始化用于 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

使用位置和暂存桶初始化 Python 的 Vertex AI SDK。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

快速查看数据集

您使用存储在公共云存储桶中的银行营销数据集的版本，使用CSV索引文件。

将变量`IMPORT_FILE`设置为CSV索引文件在云存储中的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-tables-data/bank-marketing.csv"

通过计算CSV索引文件中的行数来统计示例数量（`wc -l`），然后查看前几行。

在使用AutoML进行训练时，您需要知道标签列的名称，它将被保存为`label_column`。在这个数据集中，标签列被保存在最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

### 创建数据集

接下来，使用`TabularDataset`类的`create`方法创建Vertex AI托管的数据集资源，该方法接受以下参数：

- `display_name`：资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入资源。
- `bq_source`：或者，从BigQuery表中导入数据项到资源中。

此操作可能需要几分钟。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="Bank Marketing", gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

### 创建并运行训练流水线

要在您的数据集上训练AutoML模型，您需要执行以下步骤：
1）创建一个训练流水线。
2）运行该流水线。

#### 创建训练流水线

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练流水线，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `optimization_prediction_type`：训练模型的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 对于二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 对于多类分类：
    - `minimize-log-loss`
  - 对于回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是一个用于训练流水线的有向无环图（DAG）。

In [ ]:
dag = aiplatform.AutoMLTabularTrainingJob(
    display_name="bank",
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(dag)

#### 运行训练流水线

接下来，通过调用`run`方法并传入以下参数来运行DAG以启动训练作业：

- `dataset`：用于训练模型的托管数据集资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）指定的最大训练时间，以毫小时为单位（1000 = 1小时）。模型的训练成本不会超出此预算。
- `disable_early_stopping`：如果设置为`True`，则训练可能会在服务认为无法进一步改进模型目标测量时提前完成，而不用完整预算。

`run`方法完成后将返回Vertex AI模型资源。

训练流水线的执行最长需时20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="bank",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 查看模型评估分数

在模型训练完成后，您可以使用 `list_model_evaluations()` 方法查看评估分数。该方法返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

发送一个批量预测请求

向您注册的模型发送一个批量预测。

### 准备测试数据

您可以使用合成数据作为测试数据项。不要担心我们使用合成数据。

制作一个批量输入文件，之后将其存储在您的云存储桶中。与图像、视频和文本不同，表格模型的批量输入文件仅支持CSV格式。为了准备CSV文件，您需要确保：

- 第一行是带有特征（字段）名称的标题。
- 每个剩余行都是一个单独的预测请求，其对应特征值。

例如：

    "feature_1", "feature_2". ...
    value_1, value_2, ...

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 1 > tmp.csv
! gsutil cat $IMPORT_FILE | tail -n 10 >> tmp.csv

! cut -d, -f1-16 tmp.csv > batch.csv

gcs_input_uri = BUCKET_URI + "/test.csv"

! gsutil cp batch.csv $gcs_input_uri

### 发送批量解释请求

在创建模型资源后，您可以通过调用`batch_predict()`方法并使用以下参数来进行批量预测：

- `job_display_name`：批量预测任务的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `generate_explanations`：设置为`True`以生成解释。
- `sync`：如果设置为True，则调用将等待批处理作业完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="bank",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    generate_explanation=True,
    sync=False,
)

print(batch_predict_job)

###等待批量预测作业完成

接下来，等待批量作业完成。

或者，您可以在`batch_predict()`方法中将`sync`设置为`True`，以执行等待操作。

In [ ]:
batch_predict_job.wait()

获取解释

接着，从已完成的批量预测作业中获取解释结果。

结果被写入到您在批量预测请求中指定的输出 Cloud 存储位置。

调用 `iter_outputs()` 方法来获取结果中生成的每个 Cloud 存储文件的列表。每个文件都包含一个或多个解释回应，格式为指定的预测格式，即 JSONL 格式。

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

explanation_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("explanation"):
        explanation_results.append(blob.name)

tags = list()
for explanation_result in explanation_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{explanation_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

整理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于此教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
# Delete the training job
dag.delete()
# Delete the tabular dataset
dataset.delete()
# Delete the model resource
model.delete()
# Delete the batch prediction job
batch_predict_job.delete()

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

# Delete the locally generated files
! rm batch.csv tmp.csv